<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Inferência-do-INSE" data-toc-modified-id="Inferência-do-INSE-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Inferência do INSE</a></span></li><li><span><a href="#Percentual-de-evasão" data-toc-modified-id="Percentual-de-evasão-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Percentual de evasão</a></span><ul class="toc-item"><li><span><a href="#Sexo" data-toc-modified-id="Sexo-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Sexo</a></span></li><li><span><a href="#Distorção-Idade-Série" data-toc-modified-id="Distorção-Idade-Série-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Distorção Idade-Série</a></span></li><li><span><a href="#Transporte-público" data-toc-modified-id="Transporte-público-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Transporte público</a></span></li><li><span><a href="#Município-da-escola" data-toc-modified-id="Município-da-escola-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Município da escola</a></span></li><li><span><a href="#Tamanho-da-turma" data-toc-modified-id="Tamanho-da-turma-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Tamanho da turma</a></span></li></ul></li></ul></div>

In [252]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

import os
from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import pandas_profiling

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

import seaborn as sns
sns.set()

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [253]:
df = pd.read_csv(TREAT_PATH / 'analysis' / 'df_alunos.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182838 entries, 0 to 182837
Data columns (total 20 columns):
Unnamed: 0                    182838 non-null int64
ID                            182838 non-null int64
CO_ENTIDADE                   182838 non-null int64
TP_SEXO                       182838 non-null int64
NU_IDADE                      182838 non-null int64
IN_DISTORCAO                  182838 non-null int64
IN_TRANSPORTE_PUBLICO         182838 non-null int64
IN_LOCAL_ESCOLA               182838 non-null int64
N_TURMA                       182838 non-null int64
IN_EVASAO                     182838 non-null int64
NU_ANO_CENSO                  182838 non-null int64
CO_MUNICIPIO                  182838 non-null int64
IN_LABORATORIO_INFORMATICA    182838 non-null float64
IN_LABORATORIO_CIENCIAS       182838 non-null float64
IN_QUADRA_ESPORTES            182838 non-null float64
IN_BIBLIOTECA                 182838 non-null float64
IN_BANHEIRO_FORA_PREDIO       182838 non-null flo

In [254]:
df.head()

,Unnamed: 0,ID,CO_ENTIDADE,TP_SEXO,NU_IDADE,IN_DISTORCAO,IN_TRANSPORTE_PUBLICO,IN_LOCAL_ESCOLA,N_TURMA,IN_EVASAO,NU_ANO_CENSO,CO_MUNICIPIO,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BANHEIRO_FORA_PREDIO,IN_AREA_VERDE,IN_N_COMP_15,NIVEL
0,0,112132289130,33009627,2,17,0,0,2,0,0,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
1,1,113220290328,33009627,2,21,1,0,2,0,1,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
2,2,112752673352,33009627,2,16,0,0,2,0,0,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
3,3,117459545545,33009627,2,17,0,0,2,0,0,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
4,4,113868062938,33009627,2,18,1,0,2,0,1,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0


In [301]:
dic_categorias = {
    'IN_DISTORCAO': {0: 'Até 17 anos', 1: 'Maior ou igual a 17 anos'},
    'TP_SEXO': {1: 'Homem', 2: 'Mulher'},
    'IN_EVASAO': {0: 'Não evadido', 1: 'Evadido'},
    'IN_TRANSPORTE_PUBLICO': {1: 'Utiliza transporte público', 0: 'Não utiliza'},
    'IN_LOCAL_ESCOLA': {0: 'Rio de Janeiro (cidade)', 1: 'Grande Rio', 2: 'Outros municípios no estado'},
    'N_TURMA': {0: 'Menos de 30 alunos', 1: '30 a 40', 2: 'Mais de 40 alunos'}
}

In [303]:
for k,v in dic_categorias.items():
    df['CAT_'+k] = df[k].map(v)

In [305]:
df.head()

,Unnamed: 0,ID,CO_ENTIDADE,TP_SEXO,NU_IDADE,IN_DISTORCAO,IN_TRANSPORTE_PUBLICO,IN_LOCAL_ESCOLA,N_TURMA,IN_EVASAO,NU_ANO_CENSO,CO_MUNICIPIO,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BANHEIRO_FORA_PREDIO,IN_AREA_VERDE,IN_N_COMP_15,NIVEL,CAT_IN_DISTORCAO,CAT_TP_SEXO,CAT_IN_EVASAO,CAT_IN_TRANSPORTE_PUBLICO,CAT_IN_LOCAL_ESCOLA,CAT_N_TURMA
0,0,112132289130,33009627,2,17,0,0,2,0,0,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0,Até 17 anos,Mulher,Não evadido,Não utiliza,Outros municípios no estado,Menos de 30 alunos
1,1,113220290328,33009627,2,21,1,0,2,0,1,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0,Maior ou igual a 17 anos,Mulher,Evadido,Não utiliza,Outros municípios no estado,Menos de 30 alunos
2,2,112752673352,33009627,2,16,0,0,2,0,0,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0,Até 17 anos,Mulher,Não evadido,Não utiliza,Outros municípios no estado,Menos de 30 alunos
3,3,117459545545,33009627,2,17,0,0,2,0,0,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0,Até 17 anos,Mulher,Não evadido,Não utiliza,Outros municípios no estado,Menos de 30 alunos
4,4,113868062938,33009627,2,18,1,0,2,0,1,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0,Maior ou igual a 17 anos,Mulher,Evadido,Não utiliza,Outros municípios no estado,Menos de 30 alunos


##  Inferência do INSE

In [255]:
df['NIVEL'].value_counts(normalize=True) # total dos alunos

3.0    0.865687
4.0    0.123035
2.0    0.007646
5.0    0.002747
6.0    0.000590
1.0    0.000295
Name: NIVEL, dtype: float64

In [256]:
df[['CO_ENTIDADE', 'NIVEL']].drop_duplicates()['NIVEL'].value_counts(normalize=True)

3.0    0.815077
4.0    0.148410
2.0    0.024735
5.0    0.008245
1.0    0.002356
6.0    0.001178
Name: NIVEL, dtype: float64

In [257]:
df[['CO_ENTIDADE', 'NIVEL']].drop_duplicates()['NIVEL'].value_counts()

3.0    692
4.0    126
2.0     21
5.0      7
1.0      2
6.0      1
Name: NIVEL, dtype: int64

In [258]:
df.columns

Index(['Unnamed: 0', 'ID', 'CO_ENTIDADE', 'TP_SEXO', 'NU_IDADE',
       'IN_DISTORCAO', 'IN_TRANSPORTE_PUBLICO', 'IN_LOCAL_ESCOLA', 'N_TURMA',
       'IN_EVASAO', 'NU_ANO_CENSO', 'CO_MUNICIPIO',
       'IN_LABORATORIO_INFORMATICA', 'IN_LABORATORIO_CIENCIAS',
       'IN_QUADRA_ESPORTES', 'IN_BIBLIOTECA', 'IN_BANHEIRO_FORA_PREDIO',
       'IN_AREA_VERDE', 'IN_N_COMP_15', 'NIVEL'],
      dtype='object')

In [259]:
inse = df[['CO_ENTIDADE', 'IN_LOCAL_ESCOLA', 'IN_LABORATORIO_INFORMATICA', 'IN_LABORATORIO_CIENCIAS',
           'IN_QUADRA_ESPORTES', 'IN_BIBLIOTECA', 'IN_BANHEIRO_FORA_PREDIO', 'IN_AREA_VERDE', 
           'IN_N_COMP_15', 'NIVEL']].drop_duplicates()
inse.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1097 entries, 0 to 34258
Data columns (total 10 columns):
CO_ENTIDADE                   1097 non-null int64
IN_LOCAL_ESCOLA               1097 non-null int64
IN_LABORATORIO_INFORMATICA    1097 non-null float64
IN_LABORATORIO_CIENCIAS       1097 non-null float64
IN_QUADRA_ESPORTES            1097 non-null float64
IN_BIBLIOTECA                 1097 non-null float64
IN_BANHEIRO_FORA_PREDIO       1097 non-null float64
IN_AREA_VERDE                 1097 non-null float64
IN_N_COMP_15                  1097 non-null int64
NIVEL                         849 non-null float64
dtypes: float64(7), int64(3)
memory usage: 94.3 KB


In [260]:
print(pd.crosstab(inse['NIVEL'], inse['IN_LOCAL_ESCOLA']))
pd.crosstab(inse['NIVEL'], inse['IN_LOCAL_ESCOLA'], normalize=True)*100

IN_LOCAL_ESCOLA   0    1    2
NIVEL                        
1.0               0    0    2
2.0               1    1   19
3.0              81  363  248
4.0              11   45   70
5.0               0    2    5
6.0               1    0    0


IN_LOCAL_ESCOLA,0,1,2
NIVEL,,,
1.0,0.000000,0.000000,0.235571
2.0,0.117786,0.117786,2.237927
3.0,9.540636,42.756184,29.210836
4.0,1.295642,5.300353,8.244994
5.0,0.000000,0.235571,0.588928
6.0,0.117786,0.000000,0.000000


In [261]:
print(pd.crosstab(inse['NIVEL'], inse['IN_LABORATORIO_INFORMATICA']))
pd.crosstab(inse['NIVEL'], inse['IN_LABORATORIO_INFORMATICA'], normalize=1)*100

IN_LABORATORIO_INFORMATICA  0.0  1.0
NIVEL                               
1.0                           0    2
2.0                           3   18
3.0                         119  573
4.0                          17  109
5.0                           2    5
6.0                           0    1


IN_LABORATORIO_INFORMATICA,0.0,1.0
NIVEL,,
1.0,0.000000,0.282486
2.0,2.127660,2.542373
3.0,84.397163,80.932203
4.0,12.056738,15.395480
5.0,1.418440,0.706215
6.0,0.000000,0.141243


* Todas escolas fora da região metropolitana sem laboratório de informática estão no nível 3. Porém, para outros casos, > 80% também estão nesse nível

In [262]:
compare = [inse['IN_LOCAL_ESCOLA'], inse['IN_LABORATORIO_INFORMATICA']]

print(pd.crosstab(inse['NIVEL'], compare)*100)
pd.crosstab(inse['NIVEL'], compare, normalize=1)*100

IN_LOCAL_ESCOLA                0           1            2       
IN_LABORATORIO_INFORMATICA   0.0   1.0   0.0    1.0   0.0    1.0
NIVEL                                                           
1.0                            0     0     0      0     0    200
2.0                            0   100     0    100   300   1600
3.0                         1600  6500  5800  30500  4500  20300
4.0                            0  1100  1000   3500   700   6300
5.0                            0     0   100    100   100    400
6.0                            0   100     0      0     0      0


IN_LOCAL_ESCOLA                 0                     1                     2  \
IN_LABORATORIO_INFORMATICA    0.0        1.0        0.0        1.0        0.0   
NIVEL                                                                           
1.0                           0.0   0.000000   0.000000   0.000000   0.000000   
2.0                           0.0   1.282051   0.000000   0.292398   5.357143   
3.0                         100.0  83.333333  84.057971  89.181287  80.357143   
4.0                           0.0  14.102564  14.492754  10.233918  12.500000   
5.0                           0.0   0.000000   1.449275   0.292398   1.785714   
6.0                           0.0   1.282051   0.000000   0.000000   0.000000   

IN_LOCAL_ESCOLA                        
IN_LABORATORIO_INFORMATICA        1.0  
NIVEL                                  
1.0                          0.694444  
2.0                          5.555556  
3.0                         70.486111  
4.0                         21.875000  
5.0                          1.388889  
6.0                          0.000000

In [263]:
print(pd.crosstab(inse['NIVEL'], inse['IN_LABORATORIO_CIENCIAS']))
pd.crosstab(inse['NIVEL'], inse['IN_LABORATORIO_CIENCIAS'], normalize=1)*100

IN_LABORATORIO_CIENCIAS  0.0  1.0
NIVEL                            
1.0                        2    0
2.0                       11   10
3.0                      174  518
4.0                       31   95
5.0                        1    6
6.0                        0    1


IN_LABORATORIO_CIENCIAS,0.0,1.0
NIVEL,,
1.0,0.913242,0.000000
2.0,5.022831,1.587302
3.0,79.452055,82.222222
4.0,14.155251,15.079365
5.0,0.456621,0.952381
6.0,0.000000,0.158730


In [264]:
compare = [inse['IN_LABORATORIO_INFORMATICA'], inse['IN_LABORATORIO_CIENCIAS']]
pd.crosstab(inse['NIVEL'], compare, normalize=1)*100

IN_LABORATORIO_INFORMATICA        0.0                   1.0           
IN_LABORATORIO_CIENCIAS           0.0        1.0        0.0        1.0
NIVEL                                                                 
1.0                          0.000000   0.000000   1.075269   0.000000
2.0                          9.090909   0.000000   4.301075   1.915709
3.0                         75.757576  87.037037  80.107527  81.226054
4.0                         15.151515  11.111111  13.978495  15.900383
5.0                          0.000000   1.851852   0.537634   0.766284
6.0                          0.000000   0.000000   0.000000   0.191571

In [265]:
pd.crosstab(inse['NIVEL'], inse['IN_BIBLIOTECA'], normalize=1)*100

IN_BIBLIOTECA,0.0,1.0
NIVEL,,
1.0,1.428571,0.128370
2.0,2.857143,2.439024
3.0,84.285714,81.258023
4.0,10.000000,15.275995
5.0,1.428571,0.770218
6.0,0.000000,0.128370


In [266]:
compare = [inse['IN_BIBLIOTECA'], inse['IN_LABORATORIO_CIENCIAS']]
pd.crosstab(inse['NIVEL'], compare, normalize=1)*100

IN_BIBLIOTECA             0.0              1.0           
IN_LABORATORIO_CIENCIAS   0.0   1.0        0.0        1.0
NIVEL                                                    
1.0                       2.0   0.0   0.591716   0.000000
2.0                       4.0   0.0   5.325444   1.639344
3.0                      86.0  80.0  77.514793  82.295082
4.0                       8.0  15.0  15.976331  15.081967
5.0                       0.0   5.0   0.591716   0.819672
6.0                       0.0   0.0   0.000000   0.163934

In [267]:
pd.crosstab(inse['NIVEL'], inse['IN_BANHEIRO_FORA_PREDIO'], normalize=1)*100

IN_BANHEIRO_FORA_PREDIO,0.0,1.0
NIVEL,,
1.0,1.020408,0.000000
2.0,2.551020,2.450230
3.0,77.040816,82.848392
4.0,17.857143,13.935681
5.0,1.020408,0.765697
6.0,0.510204,0.000000


In [268]:
compare = [inse['IN_BIBLIOTECA'], inse['IN_BANHEIRO_FORA_PREDIO']]
pd.crosstab(inse['NIVEL'], compare, normalize=1)*100

IN_BIBLIOTECA                  0.0              1.0           
IN_BANHEIRO_FORA_PREDIO        0.0   1.0        0.0        1.0
NIVEL                                                         
1.0                       2.222222   0.0   0.662252   0.000000
2.0                       2.222222   4.0   2.649007   2.388535
3.0                      88.888889  76.0  73.509934  83.121019
4.0                       4.444444  20.0  21.854305  13.694268
5.0                       2.222222   0.0   0.662252   0.796178
6.0                       0.000000   0.0   0.662252   0.000000

In [269]:
pd.crosstab(inse['NIVEL'], inse['IN_N_COMP_15'], normalize=1)*100

IN_N_COMP_15,0,1
NIVEL,,
1.0,0.313480,0.000000
2.0,2.664577,1.895735
3.0,81.347962,81.990521
4.0,14.576803,15.639810
5.0,1.097179,0.000000
6.0,0.000000,0.473934


In [270]:
compare = [inse['IN_LABORATORIO_INFORMATICA'], inse['IN_LABORATORIO_CIENCIAS'], inse['IN_N_COMP_15']]
           # inse['IN_BIBLIOTECA'], inse['IN_BANHEIRO_FORA_PREDIO'], inse['IN_QUADRA_ESPORTES']]

pd.crosstab(inse['NIVEL'], compare, normalize=1)*100

IN_LABORATORIO_INFORMATICA     0.0                                 1.0  \
IN_LABORATORIO_CIENCIAS        0.0               1.0               0.0   
IN_N_COMP_15                     0      1          0      1          0   
NIVEL                                                                    
1.0                          0.000    0.0   0.000000    0.0   1.526718   
2.0                          9.375    0.0   0.000000    0.0   5.343511   
3.0                         78.125    0.0  86.792453  100.0  79.389313   
4.0                         12.500  100.0  11.320755    0.0  12.977099   
5.0                          0.000    0.0   1.886792    0.0   0.763359   
6.0                          0.000    0.0   0.000000    0.0   0.000000   

IN_LABORATORIO_INFORMATICA                                   
IN_LABORATORIO_CIENCIAS                      1.0             
IN_N_COMP_15                        1          0          1  
NIVEL                                                        
1.0                          0.000000   0.000000   0.000000  
2.0                          1.818182   1.897019   1.960784  
3.0                         81.818182  80.758808  82.352941  
4.0                         16.363636  16.260163  15.032680  
5.0                          0.000000   1.084011   0.000000  
6.0                          0.000000   0.000000   0.653595

In [271]:
compare = [inse['IN_BIBLIOTECA'], inse['IN_BANHEIRO_FORA_PREDIO'], inse['IN_QUADRA_ESPORTES']]

pd.crosstab(inse['NIVEL'], compare, normalize=1)*100

IN_BIBLIOTECA                  0.0                                    1.0  \
IN_BANHEIRO_FORA_PREDIO        0.0              1.0                   0.0   
IN_QUADRA_ESPORTES             0.0        1.0   0.0        1.0        0.0   
NIVEL                                                                       
1.0                       5.555556   0.000000   0.0   0.000000   0.000000   
2.0                       5.555556   0.000000  25.0   0.000000   2.631579   
3.0                      77.777778  96.296296  50.0  80.952381  73.684211   
4.0                       5.555556   3.703704  25.0  19.047619  21.052632   
5.0                       5.555556   0.000000   0.0   0.000000   2.631579   
6.0                       0.000000   0.000000   0.0   0.000000   0.000000   

IN_BIBLIOTECA                                             
IN_BANHEIRO_FORA_PREDIO                   1.0             
IN_QUADRA_ESPORTES             1.0        0.0        1.0  
NIVEL                                                     
1.0                       0.884956   0.000000   0.000000  
2.0                       2.654867   7.142857   2.280130  
3.0                      73.451327  71.428571  83.387622  
4.0                      22.123894  21.428571  13.517915  
5.0                       0.000000   0.000000   0.814332  
6.0                       0.884956   0.000000   0.000000

In [272]:
pd.crosstab(inse['NIVEL'], compare)

IN_BIBLIOTECA           0.0             1.0             
IN_BANHEIRO_FORA_PREDIO 0.0     1.0     0.0     1.0     
IN_QUADRA_ESPORTES      0.0 1.0 0.0 1.0 0.0 1.0 0.0  1.0
NIVEL                                                   
1.0                       1   0   0   0   0   1   0    0
2.0                       1   0   1   0   1   3   1   14
3.0                      14  26   2  17  28  83  10  512
4.0                       1   1   1   4   8  25   3   83
5.0                       1   0   0   0   1   0   0    5
6.0                       0   0   0   0   0   1   0    0

## Percentual de evasão

In [306]:
v = df['CAT_IN_EVASAO'].value_counts(normalize=True)

fig = v.iplot(kind='bar', color='#8BD9CA', title='Percentual de alunos do 1º ano evadidos e não evadidos (2016 para 2017)')

### Sexo

In [43]:
# def plot_perc_categoria(x):
    
#     v = df[['ID', 'IN_EVASAO', x]].groupby(['IN_EVASAO', x]).sum()
#     return v

In [214]:
colorscale = ['#025951', '#8BD9CA', '#BF7F30', '#F2C124', '#8C470B', '#DFC27D']

In [313]:
def plot_dist(x, title2):
    
    v_cat = df[x].value_counts(normalize=True)
    data = [go.Bar({'x': v_cat.index, 'y': v_cat, 'marker': {'color': colorscale}})]

    plotly.offline.iplot({'data': data, 'layout': {'title': title2}}, image='png', filename='dist_'+x)

def plot_categoria(x, title1, filename, ambos=True, title2=None):
    
    if ambos:
        # Distribuição total
        plot_dist(x, title2)
        
    # Distribuição com evasão
    data = []
    i = 0
    for label in df[x].unique():
        
        v_cat = df[df[x] == label]['CAT_IN_EVASAO'].value_counts(normalize=True)

        data.append(go.Bar({'x': v_cat.index, 'y': v_cat, 'name': label,
                           'marker': {'color': colorscale[i]}}))
        i += 1
        
    data.append(go.Bar({'x': v.index, 'y': v, 'name': 'Total', 
                        'marker': {'color': '#dbdbdb'}}))

    plotly.offline.iplot({'data': data, 'layout': {'title': title1}}, 
                         image='png', filename=filename)

In [315]:
title = 'Dentre as mulheres e homens, qual percentual evade?'
x = 'CAT_TP_SEXO'
categorias = {1: 'Homens', 2: 'Mulheres'}

plot_categoria(x, title, 'perc_sexo_evasao', title2='Distribuição de sexo entre alunos')

In [53]:
# v_sexo = df[df['TP_SEXO'] == 1]['IN_EVASAO'].value_counts(normalize=True)

# v_sexo.index = v_sexo.index.map({1: 'Evadidos', 0: 'Não evadidos'})

# data = [go.Bar({'x': v_sexo.index, 
#                 'y': v_sexo, 
#                 'name': 'Homens'}), 
#         go.Bar({'x': v.index, 
#                 'y': v, 
#                 'name': 'Total'})]

# plotly.offline.iplot({'data': data, 'layout': {'title': 'Dentre os homens, qual percentual evade?'}})

In [54]:
# v_sexo = df[['ID', 'IN_EVASAO', 'TP_SEXO']].groupby(['IN_EVASAO', 'TP_SEXO']).count() # dentre evadidos, quantas mulheres?
# n = v_sexo.groupby('IN_EVASAO').sum()

# v_sexo/n

# data = [go.Bar({'x': v_sexo.index, 
#                 'y': v_sexo, 
#                 'name': 'Mulheres'}), 
#         go.Bar({'x': v.index, 
#                 'y': v, 
#                 'name': 'Total'})]

# plotly.offline.iplot({'data': data, 'layout': {'title': 'Dentre evadidos, quantas são mulheres?'}})

### Distorção Idade-Série

In [384]:
def cross_evasao_sexo(x, y='CAT_TP_SEXO'):
    t = pd.crosstab([df[x], df[y]], df['CAT_IN_EVASAO'], normalize=0)*100
    t.columns.name = None
    return t

def plot_categoria_sexo(x, title, filename, y='CAT_TP_SEXO'):
    
    v = cross_evasao_sexo(x, y).reset_index()
        
    # Distribuição com evasão
    data = []
    i = 0
    for label in v[x].unique():

        v_cat = v[v[x] == label]
        data.append(go.Bar({'x': v_cat[y], 'y': v_cat['Evadido'], 'name': label,
                           'marker': {'color': colorscale[i]}}))
        i += 1
        
    plotly.offline.iplot({'data': data, 'layout': {'title': title}}, image='png', filename=filename)
    return v

In [374]:
title = 'Dentre alunos com e sem distorção idade-série, qual percentual evade?'
x = 'CAT_IN_DISTORCAO'

plot_categoria_sexo(x, title, 'perc_sexo_idade_evasao')

,CAT_IN_DISTORCAO,CAT_TP_SEXO,Evadido,Não evadido
0,Até 17 anos,Homem,24.445004,75.554996
1,Até 17 anos,Mulher,23.805636,76.194364
2,Maior ou igual a 17 anos,Homem,59.352009,40.647991
3,Maior ou igual a 17 anos,Mulher,56.483305,43.516695


In [246]:
title = 'Dentre alunos com e sem distorção idade-série, qual percentual evade?'
x = 'IN_DISTORCAO'
categorias = {1: 'Maior ou igual a 17 anos', 0: 'Até 17 anos'}

plot_categoria(x, categorias, title, 'perc_distorcao_evasao', title2='Percentual de alunos com e sem distorção idade-série')

### Transporte público

In [329]:
v = cross_evasao_sexo('CAT_IN_TRANSPORTE_PUBLICO').reset_index()
v

,CAT_IN_TRANSPORTE_PUBLICO,CAT_TP_SEXO,Evadido,Não evadido
0,Não utiliza,Homem,43.617775,56.382225
1,Não utiliza,Mulher,39.707073,60.292927
2,Utiliza transporte público,Homem,33.951087,66.048913
3,Utiliza transporte público,Mulher,29.814387,70.185613


In [375]:
title = 'Dentre alunos que utilizam ou não transporte público, qual percentual evade?'
x = 'CAT_IN_TRANSPORTE_PUBLICO'

plot_categoria_sexo(x, title, 'perc_sexo_transporte_evasao')

,CAT_IN_TRANSPORTE_PUBLICO,CAT_TP_SEXO,Evadido,Não evadido
0,Não utiliza,Homem,43.617775,56.382225
1,Não utiliza,Mulher,39.707073,60.292927
2,Utiliza transporte público,Homem,33.951087,66.048913
3,Utiliza transporte público,Mulher,29.814387,70.185613


In [248]:
title = 'Dentre alunos que utilizam ou não transporte público, qual percentual evade?'
x = 'IN_TRANSPORTE_PUBLICO'
categorias = {1: 'Utiliza transporte público', 0: 'Não utiliza'}

plot_categoria(x, categorias, title, 'perc_transporte_evasao', title2='Percentual de alunos por tipo de transporte')

### Município da escola

In [326]:
cross_evasao_sexo('CAT_IN_LOCAL_ESCOLA')

Evadido  Não evadido
CAT_IN_LOCAL_ESCOLA         CAT_TP_SEXO                        
Grande Rio                  Homem        34.029359    65.970641
                            Mulher       29.277211    70.722789
Outros municípios no estado Homem        35.408016    64.591984
                            Mulher       30.437691    69.562309
Rio de Janeiro (cidade)     Homem        41.643301    58.356699
                            Mulher       38.676098    61.323902

In [385]:
y = 'CAT_IN_TRANSPORTE_PUBLICO'
plot_categoria_sexo(x, title, 'perc_cross_transp_local_evasao', y)

,CAT_IN_LOCAL_ESCOLA,CAT_IN_TRANSPORTE_PUBLICO,Evadido,Não evadido
0,Grande Rio,Não utiliza,41.929600,58.070400
1,Grande Rio,Utiliza transporte público,27.654983,72.345017
2,Outros municípios no estado,Não utiliza,35.798981,64.201019
3,Outros municípios no estado,Utiliza transporte público,30.449827,69.550173
4,Rio de Janeiro (cidade),Não utiliza,47.378809,52.621191
5,Rio de Janeiro (cidade),Utiliza transporte público,37.157923,62.842077


In [250]:
title = 'Dentre os locais que os alunos estudam, qual percentual evade?'
x = 'IN_LOCAL_ESCOLA'
categorias = {0: 'Rio de Janeiro (cidade)', 1: 'Grande Rio', 2: 'Outros municípios no estado'}

plot_categoria(x, categorias, title, 'perc_cidade_evasao', title2='Percentual de alunos por município')

### Tamanho da turma

In [273]:
df['N_TURMA'].value_counts()

1    87925
0    60450
2    34463
Name: N_TURMA, dtype: int64

In [387]:
title = 'Dentre os locais que os alunos estudam, qual percentual evade?'
x = 'CAT_N_TURMA'
y = 'CAT_IN_LOCAL_ESCOLA'

plot_categoria_sexo(x, title, 'perc_cross_turma_local_evasao', y)

,CAT_N_TURMA,CAT_IN_LOCAL_ESCOLA,Evadido,Não evadido
0,30 a 40,Grande Rio,30.772744,69.227256
1,30 a 40,Outros municípios no estado,32.810494,67.189506
2,30 a 40,Rio de Janeiro (cidade),40.473286,59.526714
3,Mais de 40 alunos,Grande Rio,32.836725,67.163275
4,Mais de 40 alunos,Outros municípios no estado,35.880875,64.119125
5,Mais de 40 alunos,Rio de Janeiro (cidade),39.214347,60.785653
6,Menos de 30 alunos,Grande Rio,32.634947,67.365053
7,Menos de 30 alunos,Outros municípios no estado,32.693107,67.306893
8,Menos de 30 alunos,Rio de Janeiro (cidade),41.578947,58.421053


In [388]:
title = 'Dentre os locais que os alunos estudam, qual percentual evade?'
x = 'CAT_N_TURMA'

plot_categoria_sexo(x, title, 'perc_cross_turma_sexo_evasao')

,CAT_N_TURMA,CAT_TP_SEXO,Evadido,Não evadido
0,30 a 40,Homem,36.816307,63.183693
1,30 a 40,Mulher,32.689839,67.310161
2,Mais de 40 alunos,Homem,39.263494,60.736506
3,Mais de 40 alunos,Mulher,35.434966,64.565034
4,Menos de 30 alunos,Homem,36.822114,63.177886
5,Menos de 30 alunos,Mulher,32.024999,67.975001


In [276]:
title = 'Qual o percentual de evasão por tamanho de turma?'

x = 'N_TURMA'
categorias = {0: 'Menos de 30 alunos',  
              1: '30 a 40',
              2: 'Mais de 40 alunos'
             }

plot_categoria(x, categorias, title, 'perc_n_turma_evasao', title2='Percentual de alunos por tamanho da turma')